# Codebook of botanical RGCN project

by Qianqian Hiris Gu (qianqian.gu@nhm.ac.uk)
Natural History Museum, London

### Content
- Intro/Readme
- Data Acquisition and Preprocessing
- Heterogeneous Implementation of Relational Graph Convolution Network 
- Methodology
- Homogeneous Implementation of Relational Graph Convolution Network 

### Intro/Readme

    Knowledge graphs have become increasingly important in biodiversity research as they effectively represent, integrate, and explore complex, heterogeneous data. They help integrate diverse data sources, enabling the discovery of new insights and improving the performance of machine learning models. 

    Graph Convolution Networks (GCNs) have gained significant attention in recent years for their ability to effectively analyse and learn from graph-structured data. As an extension of GCNs, Relational Graph Convolution Networks (R-GCNs) are specifically designed to handle relational data in knowledge graphs. They can model complex relationships between entities and attributes within the graph, making them particularly useful for tasks such as link prediction, entity classification, and knowledge base completion in the digitisation pipeline.

    Some of the key features of R-GCNs include the following:

    - The ability to handle multi-relational data with various types of relationships.
    - Weight sharing across relations to reduce the number of parameters and avoid overfitting.
    - Aggregating information from neighbouring nodes with different types of relations.

    As we continue to explore the power of graph-based machine learning techniques, the R-GCN family will undoubtedly play a crucial role in advancing our understanding and ability to leverage complex relational data. R-GCNs have shown great potential in various applications, including natural language processing, recommendation systems, and species discovery. Simultaneously, R-GCNs could be improved and fine-tuned by playing some math tricks, for example, calculating attention (RGAT) or linear feature adjustment (GNN-Film).

    This Jupyter notebook provides insights into Python functions and models designed for the Planetary Knowledge Base project. Please don't hesitate to reach out if you want more info or to discuss potential re-coding/applications. I'd be happy to share additional resources and engage in a fruitful discussion.

### Data Acquisition and Preprocessing

    A graph data structure comprises nodes and edges, which defines a graph in data structures and mathematics. We generally use graphs for relationship modelling in complex scenarios. When we use a graph structure to model relationships of real-world entities, nodes can be any item ID that occupies one point of the binary relationship in a homogeneous or heterogeneous graph (no matter how complex the group relationship is, it can be decomposed into combinations of various relationships between pairs). This involves the concepts of homogeneous and heterogeneous graphs. A graph with more than two types of nodes and edges is called a heterogeneous graph, and the edges in the graph are used to represent relationships between two items in the actual modelling process.

#### Botanical knowledge graph is heterogeneous.

    In this section, my code imports required Python libraries and loads a sample CSV Data for a quick demo. The demo in this document is a featureless approach, but we can always assign feature values to either nodes or edges. For example, if we wanna give a dummy one-hot feature to a node/entity type named "catalogNumber", we can use codes like:

In [124]:
# onehot_catalogNumber = pd.get_dummies(data.catalogNumber, prefix='catalogNumber', dummy_na=True)
# node_features = onehot_catalogNumber 
# graph.nodes['catalogNumber'].data['feature'] = node_features
# might need to adjust tensor size

    We have attached a list of entities and relations of botanical specimens to provide a better understanding of GBIF data that will be used for constructing the knowledge graph.  We've also attached the raw data from Data portal and the Wikidat dump.

#### GBIF_nodes_and_relations (herbarium sheet)
https://docs.google.com/spreadsheets/d/1S4Ve2A6oy1i0iohAHZ96FgINTt1FDy4PGsVhhx43mmE/edit?usp=sharing

#### Unprocessed raw india dataset on Github
https://github.com/NaturalHistoryMuseum/rel-graph-data/blob/5194e27a3a28846dae16b62a8aec52974202f80b/india.csv.zip

#### Unprocessed Wikidata dump on Github
https://github.com/NaturalHistoryMuseum/rel-graph-data/blob/5194e27a3a28846dae16b62a8aec52974202f80b/wdump-2600.nt.gz 

In [123]:
import argparse
import torch
import torch.nn as nn
import dgl
import torch.optim as optim
from dgl.dataloading import MultiLayerFullNeighborSampler, EdgeDataLoader
from dgl.dataloading.negative_sampler import Uniform
import numpy as np
import pandas as pd
import itertools
import os
import tqdm
from dgl import save_graphs, load_graphs
import dgl.function as fn
import torch.nn.functional as F
from dgl.nn.pytorch import GraphConv, SAGEConv, HeteroGraphConv
from dgl.utils import expand_as_pair
from collections import defaultdict
import torch as th
import dgl.nn as dglnn
from dgl.data.utils import makedirs, save_info, load_info
from sklearn.metrics import roc_auc_score
import gc
gc.collect()

3289

In [45]:
# encoder to convert entity to numeric value
def encode_map(input_array):
    p_map={}
    length=len(input_array)
    for index, ele in zip(range(length),input_array):
        # print(ele,index)
        p_map[str(ele)] = index
    return p_map

# decoder to convert nodes back to entities
def decode_map(encode_map):
    de_map={}
    for k,v in encode_map.items():
        # index,ele 
        de_map[v]=k
    return de_map

In [46]:
# Function that return the encode mapping dictionary and decode mapping dictionary of an input graph
# Append the global datafram variable graph_features_pdf with the encoded mapping
def node_mapping(input_df):
    encode_map_dictionary = {}
    decode_map_dictionary = {}
    map_count = {}
    
    for (columnName, columnData) in input_df.iteritems():
        # print('Column Name : ', columnName)
        # print('Column Contents : ', len(columnData.values))
        encode_map_dictionary[columnName] = encode_map(set(input_df[columnName].values))
        decode_map_dictionary[columnName] = decode_map(encode_map_dictionary[columnName])
        
    for (columnName, columnData) in input_df.iteritems():
        graph_features_pdf[columnName + '_id_encoded'] = graph_features_pdf[columnName].apply(lambda e: encode_map_dictionary[columnName].get(str(e),-1))
        
        map_count[columnName + '_id_encoded'] =len(set(graph_features_pdf[columnName + '_id_encoded'].values))
        
    return encode_map_dictionary, decode_map_dictionary

In [47]:
file = './source_data_test.csv'  # file location

data = pd.read_csv(file,encoding='utf-8',sep=None,on_bad_lines='skip',engine='python')
columns = data.columns.tolist()   # load content into list
# print(columns)
data.head()
# print(data.head())

# Using DataFrame.copy() to create new dataframe.
test_graph = data.copy()
# print(test_graph)

# Initial a global datafram variable to store a copy of the original graph.
graph_features_pdf = test_graph

# Construct an encoded graph for tensor calculation and store the encode/decode map 
encode_dic, decode_dic = node_mapping(test_graph)
# graph_features_pdf.to_csv('graph_features_pdf.csv',index=False)
# print(graph_features_pdf)
index = int(len(graph_features_pdf.columns)/2)
final_graph_pdf = graph_features_pdf[graph_features_pdf.columns[index:len(graph_features_pdf.columns)]].sort_values(by=graph_features_pdf.columns[index], ascending=True)
# print(final_graph_pdf)

In [48]:
# src_col_list = final_graph_pdf.columns.tolist()
src_col_list = ['catalogNumber_id_encoded', 'catalogNumber_id_encoded', 'recordedBy_id_encoded']
dst_col_list = ['country_id_encoded', 'recordedBy_id_encoded', 'country_id_encoded']

# specimen located in country
located_in_src = final_graph_pdf[src_col_list[0]].values
located_in_dst = final_graph_pdf[dst_col_list[0]].values
# specimen recorded_by collector
recorded_by_src = final_graph_pdf[src_col_list[1]].values
recorded_by_dst = final_graph_pdf[dst_col_list[1]].values
# collector travelled to country 
travelled_to_src = final_graph_pdf[src_col_list[2]].values
travelled_to_dst = final_graph_pdf[dst_col_list[2]].values

# label is not necessary in unsupervised learning  
# catalog_node_buy_label = final_graph_pdf['label'].values

# Construct the hetreograph with specified nodes and relations. 
# Nodes and relations could imported from separate csv using pd.read_csv
# The botanical knowledge graph should be bidirectional
# The demo here assume all nodes connected with valid info
hetero_graph = dgl.heterograph({
    (src_col_list[0], 'located_in', dst_col_list[0]): (located_in_src, located_in_dst),
    (dst_col_list[0], 'bi_located_in', src_col_list[0]): (located_in_dst, located_in_src),
    (src_col_list[1], 'recorded_by', dst_col_list[1]): (recorded_by_src, recorded_by_dst),
    (dst_col_list[1], 'bi_recorded_by', src_col_list[1]): (recorded_by_dst, recorded_by_src),
    (src_col_list[2], 'travelled_to', dst_col_list[2]): (travelled_to_src, travelled_to_dst),
    (dst_col_list[2], 'bi_travelled_to', src_col_list[2]): (travelled_to_dst, travelled_to_src)
})

print(hetero_graph)

Graph(num_nodes={'catalogNumber_id_encoded': 52091, 'country_id_encoded': 10, 'recordedBy_id_encoded': 2702},
      num_edges={('catalogNumber_id_encoded', 'located_in', 'country_id_encoded'): 86782, ('catalogNumber_id_encoded', 'recorded_by', 'recordedBy_id_encoded'): 86782, ('country_id_encoded', 'bi_located_in', 'catalogNumber_id_encoded'): 86782, ('country_id_encoded', 'bi_travelled_to', 'recordedBy_id_encoded'): 86782, ('recordedBy_id_encoded', 'bi_recorded_by', 'catalogNumber_id_encoded'): 86782, ('recordedBy_id_encoded', 'travelled_to', 'country_id_encoded'): 86782},
      metagraph=[('catalogNumber_id_encoded', 'country_id_encoded', 'located_in'), ('catalogNumber_id_encoded', 'recordedBy_id_encoded', 'recorded_by'), ('country_id_encoded', 'catalogNumber_id_encoded', 'bi_located_in'), ('country_id_encoded', 'recordedBy_id_encoded', 'bi_travelled_to'), ('recordedBy_id_encoded', 'catalogNumber_id_encoded', 'bi_recorded_by'), ('recordedBy_id_encoded', 'country_id_encoded', 'travell

In [49]:
# specimen located in country
located_in_count = len(located_in_dst)
print("located_in_count", located_in_count)

# specimen recorded_by collector
recorded_by_count = len(recorded_by_dst)
print("recorded_by_count", recorded_by_count)

located_in_count 86782
recorded_by_count 86782


### Heterogeneous Implementation of Relational Graph Convolution Network

    This section will show the model logic with a simple demo code.

    In heterogeneous graphs for link prediction, we use the embedding information of the nodes on both sides of the edge and judge the existence or non-existence of the edge based on factors such as similarity and relevance. It is worth noting that although the information of the two nodes on both sides of the edge is used, from the perspective of multiple rounds of training, these two nodes also integrate the local structure and global properties of the surrounding nodes and make judgments based on the spatial structure of the graph. We need to sample negative edges for link prediction. Positive samples are constructed based on the existing node relationships, while negative samples are composed of randomly sampled edges in the graph. The model training is based on the loss constructed from the homophily assumption that nodes with closer proximity have similar characteristics. This can be considered as the essence of link (relationship) prediction in graphs.

    GCN series models are spatial structures that are non-Euclidean structures. Every node has its embedding in GNN or GCN series models. Each copies its information along the edge through a message-passing process into the neighbour node's mailbox. Then, each node can obtain the embeddings sent by its neighbours from its mailbox and somehow aggregate its embedding and the neighbour node's embeddings. Standard aggregation methods include Mean and Max operations, among others. Furthermore, since the information from multiple neighbours forms a sequence, we can use RNN, LSTM, or similar approaches for aggregation.

Heterogeneous graph convolutional layer model calls HeteroGraphConv function to convolute various relationships and then fuse nodes of the same type. In the graph convolution, there are model parameters such as weights. If these weights are not passed from the outside, creating them within the convolution essentially adds a fully connected layer to the model. This allows the convolution to be computed for each type of edge within the graph.

In [125]:
class RelGraphConvLayer(nn.Module):

    def __init__(self,
                 in_feat,
                 out_feat,
                 rel_names,
                 num_bases,
                 *,
                 weight=True,
                 bias=True,
                 activation=None,
                 regularizer="bdd",
                 self_loop=False,
                 dropout=0.2):
        
        super(RelGraphConvLayer, self).__init__()
        self.in_feat = in_feat
        self.out_feat = out_feat
        self.rel_names = rel_names
        self.num_bases = num_bases
        self.bias = bias
        self.activation = activation
        self.regularizer = regularizer
        self.self_loop = self_loop
        
        # Only for computation purposes and does not store data. 
        self.conv = HeteroGraphConv({
            rel: GraphConv(in_feat, out_feat, norm='right', weight=False, bias=False)
            for rel in rel_names
        })

        self.use_weight = weight
        self.use_basis = num_bases < len(self.rel_names) and weight
        if self.use_weight:
            if self.use_basis:
                self.basis = dglnn.WeightBasis((in_feat, out_feat), num_bases, len(self.rel_names))
            else:
                # Each relation, set another weight, fully connected layer
                self.weight = nn.Parameter(th.Tensor(len(self.rel_names), in_feat, out_feat))
                nn.init.xavier_uniform_(self.weight, gain=nn.init.calculate_gain('relu'))

        # bias
        if bias:
            self.h_bias = nn.Parameter(th.Tensor(out_feat))
            nn.init.zeros_(self.h_bias)

        # weight for self loop
        if self.self_loop:
            self.loop_weight = nn.Parameter(th.Tensor(in_feat, out_feat))
            nn.init.xavier_uniform_(self.loop_weight,
                                    gain=nn.init.calculate_gain('relu'))

        self.dropout = nn.Dropout(dropout)

    def forward(self, g, inputs):
        # Each relation has a weight matrix corresponding to the input dimension and output dimension
        g = g.local_var()
        if self.use_weight:
            weight = self.basis() if self.use_basis else self.weight
            wdict = {self.rel_names[i]: {'weight': w.squeeze(0)}
                     for i, w in enumerate(th.split(weight, 1, dim=0))}
        else:
            wdict = {}

        if g.is_block:
            inputs_src = inputs
            inputs_dst = {k: v[:g.number_of_dst_nodes(k)] for k, v in inputs.items()}
        else:
            inputs_src = inputs_dst = inputs

        # Output of the multi-type edge node convolution 
        # Input: blocks, embeding
        hs = self.conv(g, inputs, mod_kwargs=wdict)

        def _apply(ntype, h):
            if self.self_loop:
                h = h + th.matmul(inputs_dst[ntype], self.loop_weight)
            if self.bias:
                h = h + self.h_bias
            if self.activation:
                h = self.activation(h)
            return self.dropout(h)

        #
        return {ntype: _apply(ntype, h) for ntype, h in hs.items()}

In [126]:
"""
This python class only returns a dictionary, but this dictionary includes multiple Embeding Variables. 
Note that the Variables here can all be updated as the network training changes. 
We can get the Embeding of the corresponding element according to the node type and node ID
"""
class RelGraphEmbed(nn.Module):
    r"""Embedding layer for featureless heterograph."""

    def __init__(self,
                 g,
                 embed_size,
                 embed_name='embed',
                 activation=None,
                 dropout=0.0):
        super(RelGraphEmbed, self).__init__()
        self.g = g
        self.embed_size = embed_size
        self.embed_name = embed_name
        self.activation = activation
        self.dropout = nn.Dropout(dropout)

        # create weight embeddings for each node for each relation
        self.embeds = nn.ParameterDict()
        for ntype in g.ntypes:
            embed = nn.Parameter(torch.Tensor(g.number_of_nodes(ntype), self.embed_size))
            nn.init.xavier_uniform_(embed, gain=nn.init.calculate_gain('relu'))
            self.embeds[ntype] = embed

    def forward(self, block=None):
        
        return self.embeds

In [127]:
"""
The RGCN structure that includes the forward method for training and the inference method
The inference method can be used to export the embedding of each node
"""
class EntityClassify(nn.Module):
    def __init__(self,
                 g,
                 h_dim, out_dim,
                 num_bases=-1,
                 num_hidden_layers=1,
                 dropout=0,
                 use_self_loop=False):
        super(EntityClassify, self).__init__()
        self.g = g
        self.h_dim = h_dim
        self.out_dim = out_dim
        self.rel_names = list(set(g.etypes))
        self.rel_names.sort()
        if num_bases < 0 or num_bases > len(self.rel_names):
            self.num_bases = len(self.rel_names)
        else:
            self.num_bases = num_bases
        self.num_hidden_layers = num_hidden_layers
        self.dropout = dropout
        self.use_self_loop = use_self_loop

        self.embed_layer = RelGraphEmbed(g, self.h_dim)
        self.layers = nn.ModuleList()
        
        # i2h
        self.layers.append(RelGraphConvLayer(
            self.h_dim, self.h_dim, self.rel_names,
            self.num_bases, activation=F.relu, self_loop=self.use_self_loop,
            dropout=self.dropout, weight=False))

        # h2h , No hidden layer is added here, only 2 layers of convolution are used
        # Remove comment-out for adding hidden layer
        # for i in range(self.num_hidden_layers):
        #    self.layers.append(RelGraphConvLayer(
        #        self.h_dim, self.h_dim, self.rel_names,
        #        self.num_bases, activation=F.relu, self_loop=self.use_self_loop,
        #        dropout=self.dropout))
        
        # h2o
        self.layers.append(RelGraphConvLayer(
            self.h_dim, self.out_dim, self.rel_names,
            self.num_bases, activation=None,
            self_loop=self.use_self_loop))

    # Input: blocks, embeding
    def forward(self, h=None, blocks=None):
        if h is None:
            # full graph training
            h = self.embed_layer()
        if blocks is None:
            # full graph training
            for layer in self.layers:
                h = layer(self.g, h)
        else:
            # minibatch training
            # Input: blocks, embeding
            for layer, block in zip(self.layers, blocks):
                h = layer(block, h)
        return h

    
    def inference(self, g, batch_size, device="cpu", num_workers=0, x=None):

        if x is None:
            x = self.embed_layer()

        for l, layer in enumerate(self.layers):
            y = {
                k: th.zeros(
                    g.number_of_nodes(k),
                    self.h_dim if l != len(self.layers) - 1 else self.out_dim)
                for k in g.ntypes}

            
            sampler = dgl.dataloading.MultiLayerFullNeighborSampler(1)
            dataloader = dgl.dataloading.NodeDataLoader(
                g,
                {k: th.arange(g.number_of_nodes(k)) for k in g.ntypes},
                sampler,
                batch_size = batch_size,
                shuffle = True,
                drop_last = False,
                num_workers = num_workers)
            
            for input_nodes, output_nodes, blocks in tqdm.tqdm(dataloader):
                # print(input_nodes)
                block = blocks[0].to(device)
                        
                h = {k: x[k][input_nodes[k]].to(device) for k in input_nodes.keys()}
                h = layer(block, h)

                for k in h.keys():
                    y[k][output_nodes[k]] = h[k].cpu()

            x = y
        return y


In [128]:
# Extract embedding according to node type and node ID for training update
def extract_embed(node_embed, input_nodes):
    emb = {}
    for ntype, nid in input_nodes.items():
        nid = input_nodes[ntype]
        emb[ntype] = node_embed[ntype][nid]
    return emb

    Obtaining information from neighbours to update the model might make nodes closer to each other more similar, which has both advantages and disadvantages.
    - The advantage is that it aligns with the fundamental goal of training graph models: to learn the relationships between nodes in the graph. The semantic information of these relationships is contained within the embeddings.
    - However, the disadvantage is that collecting too many neighbours during training or having too many epochs of full-graph update training may lead to weak differentiation between nodes, similar embeddings, and the over-smoothing problem.

    In more detail, training a link prediction model involves comparing the score between a seed node and a connected node and the difference between the seed node's score and the score of any other node. For example, given an edge connecting 𝑢 and 𝑣, a good model would expect the score between 𝑢 and 𝑣 to be higher than the score between 𝑢 and a node 𝑣′ sampled from an arbitrary noise distribution 𝑃𝑛(𝑣). The method of selecting neighbour nodes from an arbitrary noise distribution is called negative sampling in graph data.

Instead of using full-graph training, we import the EdgeDataLoader function to compute subgraphs with batch training.

In [129]:
neg_sample_count = 1
batch_size=500 # set batch size

sampler = MultiLayerFullNeighborSampler(2)  # sampling all nodes in second layer

"""
Batch training -- Select the positive graph composed of the edges of the seed nodes,
and the negative graph consisting of the seed nodes and their randomly sampled global nodes 
"""
hetero_graph.edges['located_in'].data['train_mask'] = torch.zeros(located_in_count, dtype=torch.bool).bernoulli(1.0)
train_ip_eids = hetero_graph.edges['located_in'].data['train_mask'].nonzero(as_tuple=True)[0]
ip_dataloader = EdgeDataLoader(
    hetero_graph, {'located_in': train_ip_eids}, sampler, negative_sampler=Uniform(neg_sample_count), batch_size=batch_size
)
hetero_graph.edges['recorded_by'].data['train_mask'] = torch.zeros(recorded_by_count, dtype=torch.bool).bernoulli(1.0)
train_item_eids = hetero_graph.edges['recorded_by'].data['train_mask'].nonzero(as_tuple=True)[0]
item_dataloader = EdgeDataLoader(
    hetero_graph, {'recorded_by': train_item_eids}, sampler, negative_sampler=Uniform(neg_sample_count), batch_size=batch_size
)

/Users/mbax9qg2/anaconda3/envs/dgl/lib/python3.9/site-packages/dgl/dataloading/dataloader.py:968: DGLWarning: EdgeDataLoader directly taking a BlockSampler will be deprecated and it will not support feature prefetching. Please use dgl.dataloading.as_edge_prediction_sampler to wrap it.
  dgl_warning(


In [130]:
# Define a Heterograph Conv model
class Model(nn.Module):

    def __init__(self, graph, hidden_feat_dim, out_feat_dim):
        super().__init__()
        self.rgcn = EntityClassify(graph,
                                   hidden_feat_dim,
                                   out_feat_dim)
        self.pred = HeteroDotProductPredictor()

    def forward(self, h, pos_g, neg_g, blocks, etype):
        h = self.rgcn(h, blocks)
        return self.pred(pos_g, h, etype), self.pred(neg_g, h, etype)


class MarginLoss(nn.Module):

    def forward(self, pos_score, neg_score):
        # Compute average of loss, changes the shape of tensor
        # 1- pos_score + neg_score
        return (1 - pos_score + neg_score.view(pos_score.shape[0], -1)).clamp(min=0).mean()


class HeteroDotProductPredictor(nn.Module):

    def forward(self, graph, h, etype):
        # update h and saved it for global
        # h contains the node representations for each edge type computed from node_clf_hetero.py
        with graph.local_scope():
            graph.ndata['h'] = h  # assigns 'h' of all node types in one shot
            graph.apply_edges(fn.u_dot_v('h', 'h', 'score'), etype=etype)
            return graph.edges[etype].data['score']

In [131]:
# in_feats = hetero_graph.nodes['type'].data['feature'].shape[1]
n_hetero_features = 16 # random set for demo
hidden_feat_dim = n_hetero_features
out_feat_dim = n_hetero_features

embed_layer = RelGraphEmbed(hetero_graph, hidden_feat_dim)
all_node_embed = embed_layer()

model = Model(hetero_graph, hidden_feat_dim, out_feat_dim)
# Parameter optimisation: weight and input embedding
all_params = itertools.chain(model.parameters(), embed_layer.parameters())
optimizer = torch.optim.Adam(all_params, lr=0.01, weight_decay=0)

loss_func = MarginLoss() # can change to self-defined loss function if needed

def train_etype_one_epoch(etype, spec_dataloader):
    losses = []
    # input_nodes is a collection of all nodes in the sampled subgraph
    for input_nodes, pos_g, neg_g, blocks in tqdm.tqdm(spec_dataloader):
        emb = extract_embed(all_node_embed, input_nodes)
        pos_score, neg_score = model(emb, pos_g, neg_g, blocks, etype)
        loss = loss_func(pos_score, neg_score)
        losses.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('{:s} Epoch {:d} | Loss {:.4f}'.format(etype, epoch, sum(losses) / len(losses)))

In [132]:
# An example to show how to run the training
for epoch in range(1):
    print("start epoch:", epoch)
    model.train()
    train_etype_one_epoch('located_in', ip_dataloader)
    train_etype_one_epoch('recorded_by', item_dataloader)

start epoch: 0


100%|███████████████████████████████████████████| 174/174 [00:36<00:00,  4.71it/s]


located_in Epoch 0 | Loss 0.1557


100%|███████████████████████████████████████████| 174/174 [00:31<00:00,  5.54it/s]

recorded_by Epoch 0 | Loss 0.1372


In [133]:
# Save graph and data to bin
save_graphs("graph.bin", [hetero_graph])
torch.save(model.state_dict(), "model.bin")

# Node embeding output by each node
col = src_col_list[0]
print("node_embed:", all_node_embed[col][0])
print(all_node_embed)

# Return logit from inference
# Set 0 to unlock
# inference_out = model.inference(hetero_graph, batch_size, 'cpu', num_workers=0, x = all_node_embed)
# print(inference_out[src_col_list[0]].shape)
# print(inference_out[src_col_list[0]][0])

node_embed: tensor([-0.0177, -0.1573, -0.0390,  0.3429, -0.1840,  0.0620, -0.0962, -0.2726,
         0.1229, -0.2841,  0.1398, -0.0286,  0.0429, -0.1783, -0.1304, -0.0655],
       grad_fn=<SelectBackward0>)
ParameterDict(
    (catalogNumber_id_encoded): Parameter containing: [torch.FloatTensor of size 52091x16]
    (country_id_encoded): Parameter containing: [torch.FloatTensor of size 10x16]
    (recordedBy_id_encoded): Parameter containing: [torch.FloatTensor of size 2702x16]
)


### Methodology

    Relational Graph Convolutional Networks (R-GCNs) are an extension of Graph Convolutional Networks (GCNs) designed to handle graphs with multiple edge types or relations, such as knowledge graphs. R-GCNs are helpful for tasks like node classification and link prediction in multi-relational graphs.
    
    Here's an overview of the methodology of R-GCNs with math equations:

#### Message-passing framework:

    R-GCNs are built on the message-passing framework, where each node aggregates messages (features) from its neighbours. In R-GCNs, the aggregation process also considers the edge types (relations) between nodes. Let G = (V, E) be a graph with nodes V and edges E. Each edge e_ij in E is associated with a relation type r ∈ R, where R is the set of all relation types in the graph. The feature matrix of nodes is represented by H^(0), where H^(0)_i is the feature vector of node i.

#### R-GCN layer:

    In an R-GCN layer, each node updates its features by aggregating information from its neighbours, considering the relation types between nodes. The updated features H^(l+1)_i for node i in layer l+1 are computed as follows:
    
    H^(l+1)i = σ(∑(j, r)∈N(i) (W_r^(l) H^(l)_j))
    
    Here, N(i) represents the set of neighbour nodes of node i, considering the relation type r. W_r^(l) is the weight matrix for relation type r in layer l, and σ is a non-linear activation function, such as ReLU.

#### Weight sharing:

    R-GCNs often use weight sharing or regularisation techniques, such as block-diagonal decomposition and basis decomposition, to handle many relation types and reduce the number of parameters in the model. 
    
    Block-diagonal decomposition refers to a way of expressing a matrix as a block-diagonal form, where each block is a square matrix that corresponds to a specific subspace of the matrix.  In other words, the matrix is decomposed into a sum of square matrices that are each confined to a certain subset of rows and columns.  This decomposition can be useful for solving systems of equations or diagonalising matrices that are not necessarily diagonal or triangular.
    
    Basis decomposition, on the other hand, refers to a way of expressing a matrix as a linear combination of basis matrices, where the basis matrices are chosen to capture the important features or patterns in the matrix.  In other words, the matrix is decomposed into a sum of basis matrices that can be used to reconstruct the original matrix with minimal error.
    Basis decomposition represents each relation-specific weight matrix W_r as a linear combination of basis weight matrices B_k:
    
    W_r = ∑_k α_rk B_k
    
    Here, α_rk is a scalar weight for basis matrix B_k and relation type r. This decomposition reduces the number of parameters in the model, mitigating overfitting and improving generalisation.
    

#### Final model:
    An R-GCN model typically comprises multiple R-GCN layers, followed by a task-specific output layer. For node classification, a softmax output layer can be used, while for link prediction, a scoring function, such as DistMult or ComplEx, can be applied to compute the likelihood of potential links.

    For training, a loss function specific to the task (e.g., cross-entropy loss for node classification or ranking loss for link prediction) is used. The model parameters are optimised using gradient-based optimisation techniques.

### Homogeneous Implementation of Relational Graph Convolution Network

This implementation is an extended version based on the DGL built-in RGCN model and the official implementation (https://github.com/tkipf/relational-gcn). Details can be checked in the code comments.

In [137]:
# Converting the hetero graph into homogeneous 
# https://docs.dgl.ai/en/0.9.x/generated/dgl.to_homogeneous.html
test_homograph = dgl.to_homogeneous(hetero_graph)
print(test_homograph)
print(test_homograph.is_homogeneous)

# Assign masks indicating whether a edge belongs to training, validation, and test set.
e_nodes = a.num_edges()
e_train = int(e_nodes * 0.6)
e_val = int(e_nodes * 0.2)
train_mask = torch.zeros(e_nodes, dtype=torch.bool)
val_mask = torch.zeros(e_nodes, dtype=torch.bool)
test_mask = torch.zeros(e_nodes, dtype=torch.bool)
train_mask[:e_train] = True
val_mask[e_train:e_train + e_val] = True
test_mask[e_train + e_val:] = True
test_homograph.edata['train_mask'] = train_mask
test_homograph.edata['val_mask'] = val_mask
test_homograph.edata['test_mask'] = test_mask

test_homograph.edata['etype'] = test_homograph.edata['_TYPE']

print(test_homograph.edata)

Graph(num_nodes=54803, num_edges=520692,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), '_TYPE': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), '_TYPE': Scheme(shape=(), dtype=torch.int64)})
True
{'_ID': tensor([    0,     1,     2,  ..., 86779, 86780, 86781]), '_TYPE': tensor([0, 0, 0,  ..., 5, 5, 5]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'etype': tensor([0, 0, 0,  ..., 5, 5, 5])}


In [138]:
# Utility function for building training and testing graphs
# A helper method that allows the knowledge graph to be devided in the subgraphs for training
def get_subset_g(g, mask, num_rels, bidirected=False):
    src, dst = g.edges()
    sub_src = src[mask]
    sub_dst = dst[mask]
    sub_rel = g.edata['etype'][mask]

    if bidirected:
        sub_src, sub_dst = th.cat([sub_src, sub_dst]), th.cat([sub_dst, sub_src])
        sub_rel = th.cat([sub_rel, sub_rel + num_rels])

    #  sub_g = dgl.graph((sub_src, sub_dst), num_nodes=g.num_nodes(), idtype=th.int32)
    sub_g = dgl.graph((sub_src, sub_dst), num_nodes=g.num_nodes())
    sub_g.edata[dgl.ETYPE] = sub_rel

    return sub_g

In [139]:
# Helper method to initialise the sets of positive samples and negative samples
class NegativeSampler:
    def __init__(self, k=10):
        self.k = k

    def sample(self, pos_samples, num_nodes):
        batch_size = len(pos_samples)
        neg_batch_size = batch_size * self.k
        neg_samples = np.tile(pos_samples, (self.k, 1))

        values = np.random.randint(num_nodes, size=neg_batch_size)
        choices = np.random.uniform(size=neg_batch_size)
        subj = choices > 0.5
        obj = choices <= 0.5
        neg_samples[subj, 0] = values[subj]
        neg_samples[obj, 2] = values[obj]
        samples = np.concatenate((pos_samples, neg_samples))

        # binary labels indicating positive and negative samples
        labels = np.zeros(batch_size * (self.k + 1), dtype=np.float32)
        labels[:batch_size] = 1

        return th.from_numpy(samples), th.from_numpy(labels)

In [140]:
# Aggregating information from node's neighbours during message-passing process.
# Global uniform for aggregating positive samples 
class GlobalUniform:
    def __init__(self, g, sample_size):
        self.sample_size = sample_size
        self.eids = np.arange(g.num_edges())

    def sample(self):
        return th.from_numpy(np.random.choice(self.eids, self.sample_size))

# Sample positive connected components by neighborhood expansions.
# Introduced in the original publication
class NeighborExpand:
    """Sample a connected component by neighborhood expansion"""
    def __init__(self, g, sample_size):
        self.g = g
        self.nids = np.arange(g.num_nodes())
        self.sample_size = sample_size

    def sample(self):
        edges = th.zeros((self.sample_size), dtype=th.int32) 
        #  int64
        neighbor_counts = (self.g.in_degrees() + self.g.out_degrees()).numpy()
        seen_edge = np.array([False] * self.g.num_edges())
        seen_node = np.array([False] * self.g.num_nodes())

        for i in range(self.sample_size):
            if np.sum(seen_node) == 0:
                node_weights = np.ones_like(neighbor_counts)
                node_weights[np.where(neighbor_counts == 0)] = 0
            else:
                # Sample a visited node if applicable.
                # This guarantees a connected component.
                node_weights = neighbor_counts * seen_node

            node_probs = node_weights / np.sum(node_weights)
            chosen_node = np.random.choice(self.nids, p=node_probs)

            # Sample a neighbor of the sampled node
            u1, v1, eid1 = self.g.in_edges(chosen_node, form='all')
            u2, v2, eid2 = self.g.out_edges(chosen_node, form='all')
            u = th.cat([u1, u2])
            v = th.cat([v1, v2])
            eid = th.cat([eid1, eid2])

            to_pick = True
            while to_pick:
                random_id = th.randint(high=eid.shape[0], size=(1,))
                chosen_eid = eid[random_id]
                to_pick = seen_edge[chosen_eid]

            chosen_u = u[random_id]
            chosen_v = v[random_id]
            edges[i] = chosen_eid
            seen_node[chosen_u] = True
            seen_node[chosen_v] = True
            seen_edge[chosen_eid] = True

            neighbor_counts[chosen_u] -= 1
            neighbor_counts[chosen_v] -= 1

        return edges

class SubgraphIterator:
    def __init__(self, g, num_rels, pos_sampler, sample_size=30000, num_epochs=6000): # 6000 epochs
        self.g = g
        self.num_rels = num_rels
        self.sample_size = sample_size
        self.num_epochs = num_epochs
        if pos_sampler == 'neighbor':
            self.pos_sampler = NeighborExpand(g, sample_size)
        else:
            self.pos_sampler = GlobalUniform(g, sample_size)
        self.neg_sampler = NegativeSampler()

    def __len__(self):
        return self.num_epochs

    def __getitem__(self, i):
        eids = self.pos_sampler.sample()
        src, dst = self.g.find_edges(eids)
        src, dst = src.numpy(), dst.numpy()
        rel = self.g.edata[dgl.ETYPE][eids.long()].numpy()

        # relabel nodes to have consecutive node IDs
        uniq_v, edges = np.unique((src, dst), return_inverse=True)
        num_nodes = len(uniq_v)
        # edges is the concatenation of src and dst with relabeled ID
        src, dst = np.reshape(edges, (2, -1))
        relabeled_data = np.stack((src, rel, dst)).transpose()

        samples, labels = self.neg_sampler.sample(relabeled_data, num_nodes)

        # use only half of the positive edges
        chosen_ids = np.random.choice(np.arange(self.sample_size),
                                      size=int(self.sample_size / 2),
                                      replace=False)
        src = src[chosen_ids]
        dst = dst[chosen_ids]
        rel = rel[chosen_ids]
        src, dst = np.concatenate((src, dst)), np.concatenate((dst, src))
        rel = np.concatenate((rel, rel + self.num_rels))
        sub_g = dgl.graph((src, dst), num_nodes=num_nodes, idtype=th.int32)
        # sub_g = dgl.graph((src, dst), num_nodes=num_nodes)
        sub_g.edata[dgl.ETYPE] = th.from_numpy(rel)
        sub_g.edata['norm'] = dgl.norm_by_dst(sub_g).unsqueeze(-1)
        uniq_v = th.from_numpy(uniq_v).view(-1).long()

        return sub_g, uniq_v, samples, labels

In [141]:
# Utility functions for evaluations (raw)
def perturb_and_get_raw_rank(emb, w, a, r, b, test_size, batch_size=100):
    """ Perturb one element in the triplets"""
    n_batch = (test_size + batch_size - 1) // batch_size
    ranks = []
    emb = emb.transpose(0, 1) # size D x V
    w = w.transpose(0, 1)     # size D x R
    for idx in range(n_batch):
        print("batch {} / {}".format(idx, n_batch))
        batch_start = idx * batch_size
        batch_end = (idx + 1) * batch_size
        batch_a = a[batch_start: batch_end]
        batch_r = r[batch_start: batch_end]
        emb_ar = emb[:,batch_a] * w[:,batch_r] # size D x E
        emb_ar = emb_ar.unsqueeze(2)           # size D x E x 1
        emb_c = emb.unsqueeze(1)               # size D x 1 x V

        # out-prod and reduce sum
        out_prod = th.bmm(emb_ar, emb_c)          # size D x E x V
        score = th.sum(out_prod, dim=0).sigmoid() # size E x V
        target = b[batch_start: batch_end]

        _, indices = th.sort(score, dim=1, descending=True)
        indices = th.nonzero(indices == target.view(-1, 1), as_tuple=False)
        ranks.append(indices[:, 1].view(-1))
    return th.cat(ranks)

# Utility functions for evaluations (filtered)
def filter(triplets_to_filter, target_s, target_r, target_o, num_nodes, filter_o=True):
    """Get candidate heads or tails to score"""
    target_s, target_r, target_o = int(target_s), int(target_r), int(target_o)

    # Add the ground truth node first
    if filter_o:
        candidate_nodes = [target_o]
    else:
        candidate_nodes = [target_s]

    for e in range(num_nodes):
        triplet = (target_s, target_r, e) if filter_o else (e, target_r, target_o)
        # Do not consider a node if it leads to a real triplet
        if triplet not in triplets_to_filter:
            candidate_nodes.append(e)
    return th.LongTensor(candidate_nodes)

def perturb_and_get_filtered_rank(emb, w, s, r, o, test_size, triplets_to_filter, filter_o=True):
    """Perturb subject or object in the triplets"""
    num_nodes = emb.shape[0]
    ranks = []
    for idx in range(test_size):
        if idx % 100 == 0:
            print("test triplet {} / {}".format(idx, test_size))
        target_s = s[idx]
        target_r = r[idx]
        target_o = o[idx]
        candidate_nodes = filter(triplets_to_filter, target_s, target_r,
                                 target_o, num_nodes, filter_o=filter_o)
        if filter_o:
            emb_s = emb[target_s]
            emb_o = emb[candidate_nodes]
        else:
            emb_s = emb[candidate_nodes]
            emb_o = emb[target_o]
        target_idx = 0
        emb_r = w[target_r]
        emb_triplet = emb_s * emb_r * emb_o
        scores = th.sigmoid(th.sum(emb_triplet, dim=1))

        _, indices = th.sort(scores, descending=True)
        rank = int((indices == target_idx).nonzero())
        ranks.append(rank)
    return th.LongTensor(ranks)

In [142]:
# Utility function to compute the Mean Reciprocal Rank (MRR)
def _calc_mrr(emb, w, test_mask, triplets_to_filter, batch_size, filter=False):
    with th.no_grad():
        test_triplets = triplets_to_filter[test_mask]
        s, r, o = test_triplets[:,0], test_triplets[:,1], test_triplets[:,2]
        test_size = len(s)

        if filter:
            metric_name = 'MRR (filtered)'
            triplets_to_filter = {tuple(triplet) for triplet in triplets_to_filter.tolist()}
            ranks_s = perturb_and_get_filtered_rank(emb, w, s, r, o, test_size,
                                                    triplets_to_filter, filter_o=False)
            ranks_o = perturb_and_get_filtered_rank(emb, w, s, r, o,
                                                    test_size, triplets_to_filter)
        else:
            metric_name = 'MRR (raw)'
            ranks_s = perturb_and_get_raw_rank(emb, w, o, r, s, test_size, batch_size)
            ranks_o = perturb_and_get_raw_rank(emb, w, s, r, o, test_size, batch_size)

        ranks = th.cat([ranks_s, ranks_o])
        ranks += 1 # change to 1-indexed
        mrr = th.mean(1.0 / ranks.float()).item()
        print("{}: {:.6f}".format(metric_name, mrr))

    return mrr

# Main evaluation function
def calc_mrr(emb, w, test_mask, triplets, batch_size=100, eval_p="filtered"):
    if eval_p == "filtered":
        mrr = _calc_mrr(emb, w, test_mask, triplets, batch_size, filter=True)
    else:
        mrr = _calc_mrr(emb, w, test_mask, triplets, batch_size)
    return mrr

class RGCN(nn.Module):
    def __init__(self, num_nodes, h_dim, out_dim, num_rels,
                 regularizer="basis", num_bases=-1, dropout=0.,
                 self_loop=False,
                 ns_mode=False):
        super(RGCN, self).__init__()

        if num_bases == -1:
            num_bases = num_rels
        self.emb = nn.Embedding(num_nodes, h_dim)
        self.conv1 = RelGraphConv(h_dim, h_dim, num_rels, regularizer,
                                  num_bases, self_loop=self_loop)
        self.conv2 = RelGraphConv(h_dim, out_dim, num_rels, regularizer, num_bases, self_loop=self_loop)
        self.dropout = nn.Dropout(dropout)
        self.ns_mode = ns_mode

    def forward(self, g, nids=None):
        if self.ns_mode:
            # forward for neighbor sampling
            x = self.emb(g[0].srcdata[dgl.NID])
            h = self.conv1(g[0], x, g[0].edata[dgl.ETYPE], g[0].edata['norm'])
            h = self.dropout(F.relu(h))
            h = self.conv2(g[1], h, g[1].edata[dgl.ETYPE], g[1].edata['norm'])
            return h
        else:
            x = self.emb.weight if nids is None else self.emb(nids)
            h = self.conv1(g, x, g.edata[dgl.ETYPE], g.edata['norm'])
            h = self.dropout(F.relu(h))
            h = self.conv2(g, h, g.edata[dgl.ETYPE], g.edata['norm'])
            return h


class LinkPredict(nn.Module):
    def __init__(self, in_dim, num_rels, h_dim=500, num_bases=100, dropout=0.2, reg_param=0.01):
        super(LinkPredict, self).__init__()
        
        # Feel free to test with different regularizers 
        # self.rgcn = RGCN(in_dim, h_dim, h_dim, num_rels * 2, regularizer="basis",
        #                  num_bases=num_bases, dropout=dropout, self_loop=True)
        
        self.rgcn = RGCN(in_dim, h_dim, h_dim, num_rels * 2, regularizer="bdd",
                          num_bases=num_bases, dropout=dropout, self_loop=True)
        self.dropout = nn.Dropout(dropout)
        self.reg_param = reg_param
        self.w_relation = nn.Parameter(th.Tensor(num_rels, h_dim))
        nn.init.xavier_uniform_(self.w_relation,
                                gain=nn.init.calculate_gain('relu'))

    def calc_score(self, embedding, triplets):
        # DistMult
        s = embedding[triplets[:,0]]
        r = self.w_relation[triplets[:,1]]
        o = embedding[triplets[:,2]]
        score = th.sum(s * r * o, dim=1)
        return score

    def forward(self, g, nids):
        return self.dropout(self.rgcn(g, nids=nids))

    def regularization_loss(self, embedding):
        return th.mean(embedding.pow(2)) + th.mean(self.w_relation.pow(2))

    def get_loss(self, embed, triplets, labels):
        # each row in the triplets is a 3-tuple of (source, relation, destination)
        score = self.calc_score(embed, triplets)
        predict_loss = F.binary_cross_entropy_with_logits(score, labels)
        reg_loss = self.regularization_loss(embed)
        return predict_loss + self.reg_param * reg_loss

In [143]:
# The actual main
if __name__ == '__main__':

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Training with RGCN module")
    model = model.to(device)
    
    graph = test_homograph # graph
    num_nodes = graph.num_nodes()
    num_rels = len(hetero_graph.etypes) # the same as data.num_rels

    train_g = get_subset_g(graph, graph.edata["train_mask"], num_rels)
    test_g = get_subset_g(graph, graph.edata["train_mask"], num_rels, bidirected=True)
    test_g.edata["norm"] = dgl.norm_by_dst(test_g).unsqueeze(-1)
    test_nids = th.arange(0, num_nodes)
    test_mask = graph.edata['test_mask']
    subg_iter = SubgraphIterator(train_g, num_rels, pos_sampler='uniform')
    dataloader = GraphDataLoader(subg_iter, batch_size=1, collate_fn=lambda x: x[0])
    
    # Prepare data for metric computation
    src, dst = graph.edges()
    triplets = th.stack([src, graph.edata['etype'], dst], dim=1)

    model = LinkPredict(num_nodes, num_rels)
    optimizer = th.optim.Adam(model.parameters(), lr=1e-2)

    

    best_mrr = 0
    test_batch_size = 5 # set to be small for testing, suggest to be around 500 for actual training
    model_state_file = 'model_state.pth'
    for epoch, batch_data in enumerate(dataloader):
        model.train()

        g, train_nids, edges, labels = batch_data
        g = g.to(device)
        train_nids = train_nids.to(device)
        edges = edges.to(device)
        labels = labels.to(device)

        embed = model(g, train_nids)
        loss = model.get_loss(embed, edges, labels)
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # clip gradients
        optimizer.step()

        print("Epoch {:04d} | Loss {:.4f} | Best MRR {:.4f}".format(epoch, loss.item(), best_mrr))

        if (epoch + 1) % test_batch_size == 0:
            # perform validation on CPU because full graph is too large
            model = model.cpu()
            model.eval()
            print("start eval")
            embed = model(test_g, test_nids)
            mrr = calc_mrr(embed, model.w_relation, test_mask, triplets,
                           batch_size=test_batch_size, eval_p='filtered')
            # save best model
            if best_mrr < mrr:
                best_mrr = mrr
                th.save({'state_dict': model.state_dict(), 'epoch': epoch}, model_state_file)

            model = model.to(device)

    print("Start testing:")
    # use best model checkpoint
    checkpoint = th.load(model_state_file)
    model = model.cpu() # test on CPU
    model.eval()
    model.load_state_dict(checkpoint['state_dict'])
    print("Using best epoch: {}".format(checkpoint['epoch']))
    embed = model(test_g, test_nids)
    calc_mrr(embed, model.w_relation, test_mask, triplets,
             batch_size=test_batch_size, eval_p=args.eval_protocol)

Training with DGL built-in RGCN module
Epoch 0000 | Loss 3.2731 | Best MRR 0.0000
Epoch 0001 | Loss 14.6303 | Best MRR 0.0000
Epoch 0002 | Loss 56.9931 | Best MRR 0.0000
Epoch 0003 | Loss 11.4010 | Best MRR 0.0000
Epoch 0004 | Loss 9.5046 | Best MRR 0.0000
start eval
test triplet 0 / 104139
test triplet 100 / 104139
test triplet 200 / 104139
test triplet 300 / 104139
test triplet 400 / 104139
test triplet 500 / 104139
test triplet 600 / 104139
test triplet 700 / 104139
test triplet 800 / 104139
test triplet 900 / 104139
test triplet 1000 / 104139
test triplet 1100 / 104139
test triplet 1200 / 104139
test triplet 1300 / 104139
test triplet 1400 / 104139


KeyboardInterrupt: 

#### Reference:
Schlichtkrull, M., Kipf, T. N., Bloem, P., van den Berg, R., Titov, I., & Welling, M. (2018). Modeling relational data with graph convolutional networks. In European Semantic Web Conference (pp. 593-607). Springer, Cham. https://doi.org/10.1007/978-3-319-93417-4_38